# Homework 4: Fun with Point Clouds?

## Problem 1: Point Cloud Fusion

### Part 1: Harris Corner Detection

#### Code Imports

In [1]:
# optional: allow Jupyter to "hot reload" the modules we import - after each change, rerun this cell (instead of restarting the kernel!!)
%load_ext autoreload
%autoreload 2

In [67]:
import glob

import numpy as np

from util import ops
from util.corner_detection import HarrisCornerDetector

### Load Data

In [3]:
rgb_paths = sorted(glob.glob("../hw4_data/problem1/rgb*"))
rgb_paths

['../hw4_data/problem1/rgb1.png',
 '../hw4_data/problem1/rgb2.png',
 '../hw4_data/problem1/rgb3.png']

In [4]:
illumination_images = [
    ops.load_image(
        path,
        return_grayscale=True,
        return_array=True
    )
    for path in rgb_paths
]

Dimensions of ../hw4_data/problem1/rgb1.png: 480 x 640
Dimensions of ../hw4_data/problem1/rgb2.png: 480 x 640
Dimensions of ../hw4_data/problem1/rgb3.png: 480 x 640


For convenience, let's map the filename of each RGB image, to the image itself.

And let's also load in the depth map images in a similar fashion:

In [5]:
depthmap_paths = sorted(glob.glob("../hw4_data/problem1/depth*"))
depthmap_paths

['../hw4_data/problem1/depth1.png',
 '../hw4_data/problem1/depth2.png',
 '../hw4_data/problem1/depth3.png']

In [6]:
depthmaps = [
    ops.load_image(
        path,
        return_grayscale=True,
        return_array=True
    )
    for path in depthmap_paths
]

Dimensions of ../hw4_data/problem1/depth1.png: 480 x 640
Dimensions of ../hw4_data/problem1/depth2.png: 480 x 640
Dimensions of ../hw4_data/problem1/depth3.png: 480 x 640


#### Determine Feature that Will Be Used to Fuse Point Clouds 

We begin by setting the hyperparameters given in in the hw 4 description:

In [7]:
derivative_operator_x = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1],
])
derivative_operator_y = derivative_operator_x.T

In [8]:
# using the Gaussian filter provided here: https://homepages.inf.ed.ac.uk/rbf/HIPR2/gsmooth.htm
gaussian_window = np.array([
    [1, 4, 7, 4, 1],
    [4, 16, 26, 16, 4],
    [7, 26, 41, 26, 7],
    [4, 16, 26, 16, 4],
    [1, 4, 7, 4, 1],
])

gaussian_window = gaussian_window * (1 / 273)

In [9]:
NUM_FEATURES_TO_SELECT = 100

Then let's go ahead and apply the Harris Corner detector:

In [18]:
path_to_image_and_corners = dict()

detector = HarrisCornerDetector()

for rgb_path, image in zip(rgb_paths, illumination_images):

    corner_response = detector.detect_features(
        image,
        use_non_max_suppression=True,
        derivative_operator_x=derivative_operator_x,
        derivative_operator_y=derivative_operator_y,
        gaussian_window=gaussian_window,
    )
    # pick top features
    top_k_points = detector.pick_top_features(
        corner_response,
        NUM_FEATURES_TO_SELECT
    )

    path_to_image_and_corners[rgb_path] = (image, top_k_points)

### Part 2: Corners to 3D Points

We begin by setting the hyperparameters given in in the hw 4 description:

In [19]:
S = 5000

In [20]:
calibration_matrix = K = np.array([
    [525.,    0, 319.5],
    [0,    525., 239.5],
    [0,       0,     1]
])

Then we can project out the 3D points corresponding to the detected corners:

In [40]:
from numpy import linalg

def convert_1_corner_to_3d(
        xy_and_depth: tuple[int, int, float],
        K: np.ndarray,
        S: int,
    ) -> np.ndarray:
    """
    Follows the equation given in the hw 4 description.

    Assumes the depth is not 0.

    Parameters:
        xy_and_depth(tuple): pixel coordinates and depth
            of a detected corner
        K(np.ndarray): 3x3 camera intrinsic matrix
        S(int)

    Returns: np.array - a row vector representing 3D worldspace
        coordinates of the corner
    """
    x, y, depth_val = xy_and_depth

    image_coordinates_homogenous = np.array([x, y, 1]).T
    worldspace_coordinates = (
        (1 / S)  * depth_val * (linalg.inv(K) @ image_coordinates_homogenous)
    )

    assert worldspace_coordinates.shape == (3,)

    return worldspace_coordinates.reshape(1, 3)


Let's apply this function across all the corners, in all the images:

In [41]:
path_to_3d_points = dict()

for image_index, image_path in enumerate(rgb_paths):

     # per image, convert appropiate corners to 3d points
     _, top_k_points = path_to_image_and_corners[image_path]
     top_k_points = top_k_points.astype(int)
     corresponding_depthmap = depthmaps[image_index]
     depth_vals = corresponding_depthmap[
          top_k_points[:, 0],
          top_k_points[:, 1]
     ].reshape(-1, 1)

     world_space_coordinates_per_image = list()

     for depth_val_index, depth_val in enumerate(depth_vals):
          if depth_val != 0:
               x, y = (
                    top_k_points[depth_val_index, 1],
                    top_k_points[depth_val_index, 0],
               )
               world_space_coord = convert_1_corner_to_3d(
                    (x, y, depth_val),
                    K,
                    S,
               )
               world_space_coordinates_per_image.append(world_space_coord)

     path_to_3d_points[image_path] = np.asarray(world_space_coordinates_per_image)


### Part 3: Corner Matching

#### Apply a Rank Transform to All of the Images

Using 5x5 windows here:

In [50]:
from util.rank_transform import RankTransform2D

In [51]:
illumination_images_rank_transformed = [
    RankTransform2D.transform(image, filter_side_length=5)
    for image in illumination_images
]

#### Data Wrangling

I have a helper function to compute the putative correspondences. But it first requires us to associate the locations of the corners with their values in the corresponding rank transformed image (which I'm also referring to as its "descriptor" below):

In [68]:
descriptors_of_all_images = dict()

for image_index, image_path in enumerate(rgb_paths):

    # per image, associate each corner location to its val in the rank transformed image
    _, top_k_points = path_to_image_and_corners[image_path]
    top_k_points = top_k_points.astype(int)
    corresponding_descriptor_image = illumination_images_rank_transformed[image_index]

    descriptors_per_image = list()

    for index_corner in np.arange(top_k_points.shape[0]):
        y, x = top_k_points[index_corner, :2]
        descriptors_per_image.append([
            y, x, np.array([corresponding_descriptor_image[y, x]])
        ])

    descriptors_of_all_images[image_path] = descriptors_per_image


#### Correspondence Search

In [69]:
from util.ops import SimilarityMeasure

In [70]:
NUM_DESIRED_MATCHES = 10  # determined by the hw 4 description

##### Images 2 and 1

In [71]:
image2_corner_descriptors = descriptors_of_all_images[rgb_paths[1]]

In [72]:
image1_corner_descriptors = descriptors_of_all_images[rgb_paths[0]]

In [73]:
image_2_to_1_top_corner_matches = HarrisCornerDetector.compute_feature_correspondences(
    (image2_corner_descriptors, image1_corner_descriptors),
    desired_num_similarities=NUM_DESIRED_MATCHES,
    similarity_metric=SimilarityMeasure.SAD,
)

##### Images 2 and 3

In [74]:
image3_corner_descriptors = descriptors_of_all_images[rgb_paths[2]]

In [75]:
image_2_to_3_top_corner_matches = HarrisCornerDetector.compute_feature_correspondences(
    (image2_corner_descriptors, image3_corner_descriptors),
    desired_num_similarities=NUM_DESIRED_MATCHES,
    similarity_metric=SimilarityMeasure.SAD,
)